###### Needed installation

In [13]:
pip install flatten_json

Note: you may need to restart the kernel to use updated packages.


##### Main libraries

In [18]:
#basic libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#scrapping libraries

import requests, json
from flatten_json import flatten # to simplify nested json files in pandas df

In [19]:
pd.set_option('max_colwidth', None)

##### 1) Scrapping upcoming events

In [20]:
# upcoming events

url = "https://betsapi2.p.rapidapi.com/v1/bet365/upcoming"

querystring = {"sport_id":"1","league_id":"10041110","page":"1"}

headers = {
    'x-rapidapi-key': "aea4b96da0mshe2e45935146cf96p11c6a9jsnc73ba2aae11c",
    'x-rapidapi-host': "betsapi2.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

###### 1.1) Upcoming events : from json to dataframe

In [21]:
upcoming_events=json.loads(response.text[67:-1])

response_columns = ['id','sport_id','time','time_status','league_id','league_name','home_id','home_name','away_id','away_name','ss','our_event_id','r_id','updated_at']
df_upc_events = pd.DataFrame(columns=response_columns)

for match in upcoming_events:
    match_list = flatten(match)
    df_upc_events = df_upc_events.append(match_list, ignore_index=True)

df_upc_events = df_upc_events.drop(['away_id', 'home_id','league_id','league_name','r_id','ss','time', 'time_status','sport_id','our_event_id','updated_at'], axis = 1)
df_upc_events = df_upc_events[0:10]

In [22]:
df_upc_events

,id,home_name,away_name
0,102540251,Levante,Cadiz
1,102540243,Celta Vigo,Real Betis
2,102540245,Eibar,Barcelona
3,102540249,Huesca,Valencia
4,102540253,Osasuna,Real Sociedad
5,102540255,Real Madrid,Villarreal
6,102540261,Valladolid,Atletico Madrid
7,102540271,Elche,Athletic Bilbao
8,102540247,Granada,Getafe
9,102540258,Sevilla,CD Alaves


##### 2) Scrapping upcoming events odds

In [23]:
url = "https://betsapi2.p.rapidapi.com/v3/bet365/prematch"

headers = {
    'x-rapidapi-key': "aea4b96da0mshe2e45935146cf96p11c6a9jsnc73ba2aae11c",
    'x-rapidapi-host': "betsapi2.p.rapidapi.com"
    }

df_upc_odds = pd.DataFrame(columns = ['id','1','X','2'])

for i in df_upc_events.id:

    querystring = {"FI":i}
    
    upcoming_odds = requests.request("GET", url, headers=headers, params=querystring)
    
    # turn output into a daframe
    
    upcoming_odds =json.loads(upcoming_odds.text[24:-2])                                    # Fitting the output
    upcoming_odds = flatten(upcoming_odds)

    FI = upcoming_odds.get('FI')
    result_1 = upcoming_odds.get('main_sp_full_time_result_odds_0_odds')                    
    result_X = upcoming_odds.get('main_sp_full_time_result_odds_1_odds')
    result_2 = upcoming_odds.get('main_sp_full_time_result_odds_2_odds')                    # Retrieving key data
    
    list = [FI,result_1,result_X, result_2]       
    a_series = pd.Series(list, index = df_upc_odds.columns)                                 # Gathering key data 
    df_upc_odds = df_upc_odds.append(a_series, ignore_index=True)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [24]:
upcoming_odds

<Response [429]>

##### 3) Merging both datasets

In [26]:
df_odds = df_upc_events.merge(df_upc_odds, how = 'left', on = 'id')

In [27]:
# To match names in the historical dataset

renaming_teams = {'CD Alaves':'Alaves','Athletic Bilbao':'Ath Bilbao','Real Betis':'Betis','Atletico Madrid':'Ath Madrid','Celta Vigo':'Celta','Real Sociedad':'Sociedad'}

df_odds['home_name'].replace(renaming_teams, inplace = True)
df_odds['away_name'].replace(renaming_teams, inplace = True)

In [28]:
df_odds.to_excel('coming_round.xlsx')